Importing libraries

In [131]:
from mesa import Agent, Model
from mesa.time import RandomActivation
import matplotlib.pyplot as plt
import random
random.seed(100)
import math

from mesa.space import SingleGrid
from mesa.space import Grid
from numpy import mean, log
from mesa.datacollection import DataCollector
#import visualization tools
from mesa.visualization.modules import CanvasGrid
from mesa.visualization.ModularVisualization import ModularServer

Agent classes:

Cooperator class: 
can directly transfer or add to community pool. If cooperator directly transacts w/ defector, cooperator turns into Neutral, and can no longer add to community pool or perform direct transactions

In [213]:
class Cooperator(Agent):
    def __init__(self, unique_id, model):
        super().__init__(unique_id, model)
        self.wealth = 5
        self.neutral = False
        self.pos = (0,0)
        self.identifier = 'C'

#cooperators only move if empty cell
    def move(self):
        current = self.pos
        possible_steps = self.model.grid.get_neighborhood(
            self.pos,
            moore=True,
            include_center=False)
        new_position = self.random.choice(possible_steps)
        if (self.model.grid.is_cell_empty(new_position)):
            self.model.grid.move_agent(self, new_position)
        
    #create wealth for community
    def add_wealth_to_community(self):
        self.model.communitypool += 1

    def take_from_communitypool(self):
        if self.wealth < 3 & self.model.communitypool > 0:
            self.model.communitypool -= 1
            self.wealth += 1

    #direct transfer of wealth btwn cooperator and other
    def direct_transfer(self):
        neighbors = self.model.grid.get_neighborhood(self.pos,
                                                     moore=False, radius=1)
        if len(neighbors) > 1:
            other = self.random.choice(neighbors)
            #if other is defector, give 1/3 money away + turn neutral
            if other.identifier == 'D':
                self.neutral = True
                self.identifier = 'N'
                transfer_amt = self.wealth / 3
                other.wealth += transfer_amt
                self.wealth = 0
                #TODO: change colors when neutral
            else:
                other.wealth += 1
                self.wealth -= 1

    def step(self):
        if self.wealth > 5:
            self.direct_transfer()
            self.move()
        elif self.wealth > 3:
            self.add_wealth_to_community()


Defector Class: defectors can take from the community pool but do not add to it. when defectors interact w/ cooperators, they turn cooperators neutral

In [133]:
class Defector(Agent):
    def __init__(self, unique_id, model):
        super().__init__(unique_id, model)
        self.wealth = 5
        self.pos = (0,0)
        self.identifier = 'D'

    def move(self):
        current = self.pos
        possible_steps = self.model.grid.get_neighborhood(
            self.pos,
            moore=True,
            include_center=False)
        new_position = self.random.choice(possible_steps)
        if (self.model.grid.is_cell_empty(new_position)):
            self.model.grid.move_agent(self, new_position)

    #at random time intervals, take from community pool
    def take_from_communitypool(self):
        if self.model.communitypool > 0:
            self.wealth += 1
            self.model.communitypool -= 1

    def step(self):
        self.move()
        #30 percent chance that defector will take from
        #community pool
        x = random.randint(0, 9)
        if x % 3 == 0:
            self.take_from_communitypool()


Community class (this is the model)

In [214]:
class Community(Model):
   # def __init__(self, C, D, width, height):
    def __init__(self, N, width, height):
        self.num_cooperators = int(N/2)
        self.num_defectors = self.num_cooperators - 2
        self.num_neutrals = 0
        self.grid = SingleGrid(width, height,torus=True)
        self.schedule = RandomActivation(self)
        self.communitypool = 0
        #create defectors
        for i in range(self.num_defectors):
            b = Defector(i, self)
            self.schedule.add(b)
           #randomly place defectors
            x = self.random.randrange(self.grid.width)
            y = self.random.randrange(self.grid.height)
            if (self.grid.is_cell_empty([x,y])):
                self.grid.place_agent(b,(x,y))
                b.pos = (x,y)
        #create cooperators
        for i in range(self.num_defectors, self.num_cooperators):
            a = Cooperator(i, self)
            self.schedule.add(a)
            #add cooperators to grid
            x = self.random.randrange(self.grid.width)
            y = self.random.randrange(self.grid.height)
            #while cell is full
            while (self.grid.is_cell_empty([x,y]) == False):
                #recalculate x and y
                x = self.random.randrange(self.grid.width)
                y = self.random.randrange(self.grid.height)
                if self.grid.is_cell_empty([x,y]):
                    break
            self.grid.position_agent(a,(x,y))
            a.pos = (x,y)
           
    def step(self):
        #update numbers of each phenotype
        for agent in self.schedule.agents:
            if agent.identifier == 'C':
                self.num_cooperators += 1
            elif agent.identifier == 'D':
                self.num_defectors += 1
            elif agent.identifier == 'N':
                self.num_cooperators -= 1
                self.num_neutrals += 1
        self.schedule.step()#update numbers of each phenotype
        



In [135]:
a = Cooperator(1, Community,)
print(a.wealth)
b = Defector(2,Community)
print(b.wealth)
model = Community(20,10,10)
print(model.communitypool)
#TODO: issue is with placing agents

5
5
0


In [136]:
model = Community(50,10,10)
print(model.num_cooperators)
print(model.num_defectors)
for i in range(20):
    model.step()

print(model.num_cooperators)
print(model.num_defectors)

25
23
65
483


Helper functions for evolving ideal community

Individual function

In [215]:
def individual():
    #Randomly initializing model values:
    #size is a random square number
    choice = random.randint(10,100)
    height = choice
    width = choice
    size = choice * choice
    #C/D is a fixed ratio of that number
    C = int(.95 * size)
    D = size - C
#create model object
    model = Community(size, width, height)
    #run model for 20 microgenerations
    for i in range(20):
        model.step()
    #capture final results of model, return list
    result = []
    result.append(model.communitypool)
    result.append(model.num_cooperators)
    result.append(model.num_defectors)
    result.append(model.num_neutrals)
    result.append(C)
    result.append(D)
    result.append(size)
    return result


In [216]:
print(individual())
#TODO: no direct transactions are occurring

[0, 2778, 57456, 0, 5202, 274, 5476]


In [222]:
#fitness function returns float
def fitness(result):
    prob_reproduction = .5
    #reward high ratio of cooperators
    if result[1] > result[2]:
        prob_reproduction = .9
    #punish high proportion of defectors
    elif result[2] > result[1]:
        prob_reproduction = 0.2
    #reward high ratio of cooperators to neutrals
    elif (result[3] / result[1]) < 0.3:
        #high probability of reproduction
        prob_reproduction = .7
    #reward arbitrarily high value for community pool
    elif result[0] > 60:
        prob_reproduction = .8
    return prob_reproduction

In [230]:
result = individual()
print(result[2])
print(result)
print(fitness(result))

98742
[0, 4744, 98742, 0, 8938, 471, 9409]
0.2


In [239]:
#mutation function randomly changes some parameter of ind
#TODO: mutations should change INITIAL conditions, not results
def mutation(individual, prob_mut=.3):
    mutated_individual = []
    if random.random() < prob_mut:
        #perform some mutation on individual
        #try: changing defectors
        mutated_individual = individual
        mutated_individual[2] = random.randint(0,10)
    else: 
        mutated_individual = individual
    return mutated_individual


Evolving model

In [277]:
list = [1,2,3,5]
list2 = [4,5,6,6]
list3 = [7,8,9,7]
list4 = []
list4.append(list)
list4.append(list2)
list4.append(list3)
print(list4[0])
pop_fitness = []
for i in range(len(list4)):
    print(list4[i])
print(list4[1])
print(fitness([1,2,3,5]))
print(fitness(list4[0]))
print(pop_fitness)
print(mutation(list, prob_mut = .5))


[1, 2, 3, 5]
[1, 2, 3, 5]
[4, 5, 6, 6]
[7, 8, 9, 7]
[4, 5, 6, 6]
0.2
0.2
[]
[1, 2, 7, 5]


In [221]:
pop_size = 100
num_generations = 500

#create population of individuals
population = []
for i in range(pop_size):
    population.append(individual())
#assign a fitness to each individual
pop_fitness = []
for i in range(len(population)):
    pop_fitness.append(fitness(population[i]))
print(pop_fitness)

[0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2]


In [278]:
#testing a new approach
pop_size = 100
num_generations = 500
population = []
for i in range(pop_size):
    ind = individual()
    mut = mutation(ind, prob_mut = .5)
    population.append(mut)
print(population)

[[0, 652, 10, 0, 1163, 62, 1225], [0, 1552, 3, 0, 2873, 152, 3025], [0, 1340, 27258, 0, 2470, 131, 2601], [0, 3652, 1, 0, 6863, 362, 7225], [0, 404, 7602, 0, 692, 37, 729], [0, 4940, 102858, 0, 9310, 491, 9801], [0, 880, 6, 0, 1596, 85, 1681], [0, 1098, 6, 0, 2010, 106, 2116], [0, 152, 2310, 0, 213, 12, 225], [0, 4272, 88830, 0, 8040, 424, 8464], [0, 202, 3360, 0, 307, 17, 324], [0, 2704, 8, 0, 5062, 267, 5329], [0, 2852, 0, 0, 5343, 282, 5625], [0, 1240, 25158, 0, 2280, 121, 2401], [0, 1444, 7, 0, 2668, 141, 2809], [0, 352, 6510, 0, 593, 32, 625], [0, 3240, 67158, 0, 6080, 320, 6400], [0, 3082, 63840, 0, 5779, 305, 6084], [0, 124, 9, 0, 160, 9, 169], [0, 1392, 28350, 0, 2568, 136, 2704], [0, 4552, 94710, 0, 8573, 452, 9025], [0, 3402, 70560, 0, 6387, 337, 6724], [0, 90, 1008, 0, 95, 5, 100], [0, 4648, 3, 0, 8755, 461, 9216], [0, 1392, 28350, 0, 2568, 136, 2704], [0, 2420, 49938, 0, 4522, 239, 4761], [0, 652, 8, 0, 1163, 62, 1225], [0, 168, 5, 0, 243, 13, 256], [0, 1444, 4, 0, 2668, 14

In [279]:
pop_size = 100
num_generations = 500

#for each generation
for generation in range(num_generations):
    #create population of individuals
    population = []
    pop_fitness = []
    #create random population w/ mutations
    for i in range(pop_size):
        ind = individual()
        mut = mutation(ind, prob_mut = .4)
        population.append(mut)
    for i in range(len(population)):
        fit = fitness(population[i])
        pop_fitness.append(fit)
    population = random.choices(population, k=pop_size, weights=pop_fitness)
    mean_fitnesses = []
    mean_fitnesses.append(mean(pop_fitness))

KeyboardInterrupt: 

Data analysis

In [ ]:
plt.figure()
plt.plot(mean_fitnesses)
plt.xlabel("Generation")
plt.ylabel("Mean Population Fitness")
plt.show()
